### To Do List
##### Done
* check all data form (sii, otc, pub, rotc)
* parsing with loop (and using tenacity)
* make it a independ .py file

##### Not Yet
* design storage structure
* build up sqlite3 db table / csv version as well
* set cron job and keep the code clean

In [2]:
import requests
import pandas as pd
import numpy as np
import math
import os
from tenacity import retry, stop_after_attempt, wait_fixed
from datetime import datetime, timedelta

In [ ]:
@retry(stop=stop_after_attempt(3), wait=wait_fixed(5))
def financial_statement(year, season, type='PL'):

    if year >= 1000:
        year -= 1911

    if type == 'PL': # 綜合損益彙總表
        url = 'https://mops.twse.com.tw/mops/web/ajax_t163sb04'
    elif type == 'BS': # 資產負債彙總表
        url = 'https://mops.twse.com.tw/mops/web/ajax_t163sb05'
    else:
        print('type does not match')
    
    df_final = pd.DataFrame()
    
    for corp_type in ["sii", "otc", "pub", "rotc"]:
        r = requests.post(url, {
            'encodeURIComponent':1,
            'step':1,
            'firstin':1,
            'off':1,
            'TYPEK':corp_type,# sii上市，otc上櫃，rotc興櫃，pub公開發行
            'year':str(year),
            'season':str(season),
        })

        r.encoding = 'utf8'
            
        dfs = pd.read_html(r.text, header=None)
        df = pd.concat(dfs[1:], axis=0, sort=False)
        df['年份'] = pd.Series([year] * df.shape[0])
        df['季度'] = pd.Series([season] * df.shape[0])
        df = pd.set_index(['公司名稱']).apply(lambda s: pd.to_numeric(s, errors='ceorce'))
        df['年份'] = pd.Series([year] * df.shape[0])
        df['季度'] = pd.Series([season] * df.shape[0])
        df['公司名稱'] = df.index
        df['公司代號'] = df['公司代號'].astype(str)
        df = df.set_index('公司代號')
        df_final = pd.concat([df_final, df], axis=0, sort=False)
            
    return df_final

@retry(stop=stop_after_attempt(3), wait=wait_fixed(5))
def financial_analysis(year, season): # 營益分析彙總表
    
    if year >= 1000:
        year -= 1911
    
    url = 'https://mops.twse.com.tw/mops/web/ajax_t163sb06'
    
    df_final = pd.DataFrame()
    
    for corp_type in ["sii", "otc", "pub", "rotc"]:
        try:
            r = requests.post(url, {
                'encodeURIComponent':1,
                'step':1,
                'firstin':1,
                'off':1,
                'TYPEK':corp_type, #otc pub rotc sii
                'year':str(year),
                'season':str(season),
            })
        
            r.encoding = 'utf8'
            dfs = pd.read_html(r.text, header=None)
            dfs[0].columns = dfs[0].iloc[0]
            df = dfs[0]
            df['年份'] = pd.Series([year] * df.shape[0])
            df['季度'] = pd.Series([season] * df.shape[0])
            df = df.set_index(['公司名稱']).apply(lambda s: pd.to_numeric(s, errors='ceorce'))
            df = df[~df['公司代號'].apply(lambda x: math.isnan(x))]
            df['公司名稱'] = df.index
            df['公司代號'] = df['公司代號'].astype(int).astype(str)
            df = df.set_index('公司代號')
            df_final = pd.concat([df_final, df], axis=0, sort=False)
        except Exception as e:
            continue

    return df_final

In [ ]:
storage = "financial_statement/"

if not os.path.exists(storage+'duration_coverage_FS.csv'):
    pd.DataFrame({'Season':[], 'Created_at':[]}).to_csv(storage+'duration_coverage_FS.csv', index=False)
existed_season = pd.read_csv(storage+'duration_coverage_FS.csv')['Season'].tolist()

df_PL = pd.read_csv('P&L.csv') if os.path.exists('P&L.csv') else pd.DataFrame()
df_BS = pd.read_csv('Balance_Sheet.csv') if os.path.exists('Balance_Sheet.csv') else pd.DataFrame()
df_FA = pd.read_csv('Financial_Analysis.csv') if os.path.exists('Financial_Analysis.csv') else pd.DataFrame()

In [ ]:
# Main Part - Start from 2013-1
for year in list(range(2013, datetime.now().year+1)):
    for season in list(range(1, 5)):
        handling_season = "{0}-{1}".format(str(year), str(season))
        
        record_str = handling_season+" - 綜合損益彙總表"

        if record_str in existed_season:
            print("Pass: ", record_str)
        else:
            print("Handling: ", record_str)
            try:
	            df_PL = pd.concat([df_PL, financial_statement(year, season, type='PL')], axis=0, sort=False)
	            duration_covered = duration_covered.append(pd.DataFrame({'Season':[record_str], 'Created_at':[datetime.now()]}), sort=True)
	            duration_covered.to_csv(storage+'duration_coverage_FS.csv', index=False)
            except Exception as e:
            	print("Failed")
        
        record_str = handling_season+" - 資產負債彙總表"

        if record_str in existed_season:
            print("Pass: ", record_str)
        else:
            print("Handling: ", record_str)
            try:
                df_BS = pd.concat([df_BS, financial_statement(year, season, type='BS')], axis=0, sort=False)
                duration_covered = duration_covered.append(pd.DataFrame({'Season':[record_str], 'Created_at':[datetime.now()]}), sort=True)
                duration_covered.to_csv(storage+'duration_coverage_FS.csv', index=False)
            except Exception as e:
                print("Failed")
                
        
        record_str = handling_season+" - 營益分析彙總表"

        if record_str in existed_season:
            print("Pass: ", record_str)
        else:
            print("Handling: ", record_str)
            try:
                df_FA = pd.concat([df_FA, financial_analysis(year, season)], axis=0, sort=False)
                duration_covered = duration_covered.append(pd.DataFrame({'Season':[record_str], 'Created_at':[datetime.now()]}), sort=True)
                duration_covered.to_csv(storage+'duration_coverage_FS.csv', index=False)
            except Exception as e:
                print("Failed")
                

In [ ]:
df_PL.to_csv(storage+'P&L.csv')
df_BS.to_csv(storage+'Balance_Sheet.csv')
df_FA.to_csv(storage+'Financial_Analysis.csv') 

In [107]:
df_FA = pd.read_csv(storage+'Financial_Analysis.csv')
df_FA.head()

,公司代號,營業收入(百萬元),毛利率(%)(營業毛利)/(營業收入),營業利益率(%)(營業利益)/(營業收入),稅前純益率(%)(稅前純益)/(營業收入),稅後純益率(%)(稅後純益)/(營業收入),年份,季度,公司名稱
0,1101,24114.05,12.95,8.40,9.71,8.29,102,1,台泥
1,1102,13931.55,6.40,2.44,11.41,9.84,102,1,亞泥
2,1103,741.19,-6.06,-20.21,4.56,8.05,102,1,嘉泥
3,1104,1248.07,9.78,2.42,14.92,14.70,102,1,環球水泥
4,1108,1203.67,12.61,8.16,7.03,5.31,102,1,幸福水泥


In [109]:
df_FA[df_FA.公司代號 == 8477]

,公司代號,營業收入(百萬元),毛利率(%)(營業毛利)/(營業收入),營業利益率(%)(營業利益)/(營業收入),稅前純益率(%)(稅前純益)/(營業收入),稅後純益率(%)(稅後純益)/(營業收入),年份,季度,公司名稱
16429,8477,1099.54,16.49,1.79,2.25,1.87,104,2,創業家
20186,8477,2417.00,16.24,1.75,2.15,1.79,104,4,創業家
22281,8477,834.06,15.44,3.40,3.72,3.09,105,1,創業家
23965,8477,1622.06,15.21,2.02,2.36,1.96,105,2,創業家
26053,8477,2317.40,15.73,1.63,1.99,1.65,105,3,創業家
27768,8477,3130.50,16.44,1.61,1.99,1.65,105,4,創業家
29885,8477,835.65,18.55,0.83,1.36,1.12,106,1,創業家
31587,8477,1720.38,17.76,0.46,0.91,0.69,106,2,創業家
33710,8477,2626.12,17.27,0.32,0.75,0.58,106,3,創業家
35416,8477,3745.81,16.40,0.24,0.65,0.50,106,4,創業家


In [95]:
#storage = "financial_statement/"
#df_PL = pd.read_csv(storage+'P&L.csv') if os.path.exists(storage+'P&L.csv') else pd.DataFrame()
#df_BS = pd.read_csv(storage+'Balance_Sheet.csv') if os.path.exists(storage+'Balance_Sheet.csv') else pd.DataFrame()
#df_FA = pd.read_csv(storage+'Financial_Analysis.csv') if os.path.exists(storage+'Financial_Analysis.csv') else pd.DataFrame()

In [4]:
df_PL.columns

Index(['公司代號', '利息淨收益', '利息以外淨損益', '呆帳費用及保證責任準備提存（各項提存）', '營業費用',
       '繼續營業單位稅前淨利（淨損）', '所得稅（費用）利益', '繼續營業單位本期稅後淨利（淨損）', '停業單位損益',
       '合併前非屬共同控制股權損益', '本期稅後淨利（淨損）', '其他綜合損益（稅後）', '合併前非屬共同控制股權綜合損益淨額',
       '本期綜合損益總額（稅後）', '淨利（損）歸屬於母公司業主', '淨利（損）歸屬於共同控制下前手權益', '淨利（損）歸屬於非控制權益',
       '綜合損益總額歸屬於母公司業主', '綜合損益總額歸屬於共同控制下前手權益', '綜合損益總額歸屬於非控制權益', '基本每股盈餘（元）',
       '收益', '支出及費用', '營業利益', '營業外損益', '稅前淨利（淨損）', '所得稅利益（費用）',
       '繼續營業單位本期淨利（淨損）', '本期淨利（淨損）', '本期其他綜合損益（稅後淨額）', '本期綜合損益總額',
       '淨利（淨損）歸屬於共同控制下前手權益', '營業收入', '營業成本', '營業毛利（毛損）', '未實現銷貨（損）益',
       '已實現銷貨（損）益', '營業毛利（毛損）淨額', '其他收益及費損淨額', '營業利益（損失）', '營業外收入及支出',
       '所得稅費用（利益）', '其他綜合損益（淨額）', '淨利（淨損）歸屬於母公司業主', '淨利（淨損）歸屬於非控制權益',
       '利息以外淨收益', '淨收益', '呆帳費用及保證責任準備提存', '保險負債準備淨變動', '繼續營業單位稅前損益',
       '繼續營業單位稅前純益（純損）', '繼續營業單位本期純益（純損）', '其他綜合損益（稅後淨額）', '收入', '支出',
       '其他綜合損益', '年份', '季度', '公司名稱', '原始認列生物資產及農產品之利益（損失）',
       '生物資產當期公允價值減出售成本之變動利益（損失）', '呆帳費用、承諾及保證責任準備提存'],
      dtype='object')

### 營業收入 = 利息淨收益 = 收益 = 收入
-營業成本  
+原始認列生物資產及農產品之利益（損失）  
+生物資產當期公允價值減出售成本之變動利益（損失）  
+*利息以外淨損益 = 利息以外淨收益* 

##### 營業毛利（毛損） 
-未實現銷貨（損）益  
+已實現銷貨（損）益  
-*呆帳費用、承諾及保證責任準備提存*  
-*保險負債準備淨變動*

### 營業毛利（毛損）淨額 = 淨收益
-營業費用 = 支出及費用 = 支出
+其他收益及費損淨額

### 營業利益（損失）= 營業利益
+營業外收入及支出 = 營業外損益

##### 稅前淨利（淨損）= 繼續營業單位稅前損益 = 繼續營業單位稅前純益（純損）
-所得稅費用（利益）= 所得稅（費用）利益

#### 繼續營業單位本期淨利（淨損） = 繼續營業單位本期純益（純損）
+停業單位損益
+合併前非屬共同控制股權損益

#### 本期淨利（淨損）= 本期稅後淨利（淨損）
+其他綜合損益（淨額）= 本期其他綜合損益（稅後淨額） = 其他綜合損益
+合併前非屬共同控制股權綜合損益淨額

### 本期綜合損益總額 = 本期綜合損益總額（稅後）

* 淨利（淨損）歸屬於母公司業主 = 淨利（損）歸屬於母公司業主
* 淨利（淨損）歸屬於共同控制下前手權益
* 淨利（淨損）歸屬於非控制權益 = 淨利（損）歸屬於非控制權益
* 綜合損益總額歸屬於母公司業主
* 綜合損益總額歸屬於共同控制下前手權益
* 綜合損益總額歸屬於非控制權益

## 基本每股盈餘（元）

In [129]:
import itertools

def checkifcombine(target, test_obj):
    for i in list(itertools.combinations(list(range(0,len(test_obj))),2)):
        if len(target[(target[test_obj[i[0]]].notnull() & target[test_obj[i[1]]].notnull())]) > 0:
            print(test_obj[i[0]], "&", test_obj[i[1]], "-> Cannot be combined")
            return False
    print(str(test_obj), "is OK to be compined")
    return True

def combine_same_column(df, target_col, combined_list):
    l = combined_list + target_col
    if checkifcombine(df, l):
        df[target_col[0]] = df[l].sum(axis=1)
        return df.drop(columns=combined_list)
    else:
        return

def PL_handling(df_PL):
    storage = "financial_statement/"
    if os.path.exists(storage+'P&L.csv'):
        df_PL = pd.read_csv(storage+'P&L.csv')
        print("Before: ", df_PL.shape)
    else:
        return("No P&L.csv file")
    df_PL = df_PL.replace({'0':np.nan, 0:np.nan, 0.0:np.nan})
    df_PL = combine_same_column(df_PL, ['營業收入'], ['利息淨收益', '收益', '收入'])
    df_PL = combine_same_column(df_PL, ['利息以外淨損益'], ['利息以外淨收益'])
    df_PL = combine_same_column(df_PL, ['營業毛利（毛損）淨額'], ['淨收益'])
    df_PL = combine_same_column(df_PL, ['營業費用'], ['支出及費用', '支出'])
    df_PL = combine_same_column(df_PL, ['營業利益'], ['營業利益（損失）'])
    df_PL = combine_same_column(df_PL, ['營業外損益'], ['營業外收入及支出'])
    df_PL = combine_same_column(df_PL, ['稅前淨利（淨損）'], ['繼續營業單位稅前損益', '繼續營業單位稅前純益（純損）'])         
    df_PL = combine_same_column(df_PL, ['所得稅費用（利益）'], ['所得稅（費用）利益'])
    df_PL = combine_same_column(df_PL, ['其他綜合損益'], ['本期其他綜合損益（稅後淨額）', '其他綜合損益（淨額）'])
    df_PL = combine_same_column(df_PL, ['本期淨利（淨損）'], ['本期稅後淨利（淨損）'])
    df_PL = combine_same_column(df_PL, ['繼續營業單位本期淨利（淨損）'], ['繼續營業單位本期純益（純損）'])
    df_PL = combine_same_column(df_PL, ['本期綜合損益總額'], ['本期綜合損益總額（稅後）'])
    df_PL = combine_same_column(df_PL, ['淨利（淨損）歸屬於母公司業主'], ['淨利（損）歸屬於母公司業主'])
    df_PL = combine_same_column(df_PL, ['淨利（淨損）歸屬於非控制權益'], ['淨利（損）歸屬於非控制權益'])
    print("After: ", df_PL.shape)
    return df_PL

df_PL.to_csv(storage+'P&L_clean.csv', index=False)
df_BS = df_BS.drop(columns=['Unnamed: 13','Unnamed: 12'])
df_BS.to_csv(storage+'Balance_Sheet.csv')

In [120]:
df_PL.columns

Index(['公司代號', '利息以外淨損益', '呆帳費用及保證責任準備提存（各項提存）', '營業費用', '繼續營業單位稅前淨利（淨損）',
       '繼續營業單位本期稅後淨利（淨損）', '停業單位損益', '合併前非屬共同控制股權損益', '其他綜合損益（稅後）',
       '合併前非屬共同控制股權綜合損益淨額', '淨利（損）歸屬於共同控制下前手權益', '綜合損益總額歸屬於母公司業主',
       '綜合損益總額歸屬於共同控制下前手權益', '綜合損益總額歸屬於非控制權益', '基本每股盈餘（元）', '營業利益', '營業外損益',
       '稅前淨利（淨損）', '所得稅利益（費用）', '繼續營業單位本期淨利（淨損）', '本期淨利（淨損）', '本期綜合損益總額',
       '淨利（淨損）歸屬於共同控制下前手權益', '營業收入', '營業成本', '營業毛利（毛損）', '未實現銷貨（損）益',
       '已實現銷貨（損）益', '營業毛利（毛損）淨額', '其他收益及費損淨額', '所得稅費用（利益）', '淨利（淨損）歸屬於母公司業主',
       '淨利（淨損）歸屬於非控制權益', '呆帳費用及保證責任準備提存', '保險負債準備淨變動', '其他綜合損益（稅後淨額）',
       '其他綜合損益', '年份', '季度', '公司名稱', '原始認列生物資產及農產品之利益（損失）',
       '生物資產當期公允價值減出售成本之變動利益（損失）', '呆帳費用、承諾及保證責任準備提存'],
      dtype='object')

In [128]:
for i in df_BS.columns:
    print(i)
df_BS[df_BS['Unnamed: 13'].notnull()]

公司代號
現金及約當現金
存放央行及拆借銀行同業
透過損益按公允價值衡量之金融資產
避險之衍生金融資產淨額
附賣回票券及債券投資淨額
應收款項－淨額
當期所得稅資產
待出售資產－淨額
貼現及放款－淨額
備供出售金融資產－淨額
持有至到期日金融資產－淨額
採用權益法之投資－淨額
受限制資產－淨額
其他金融資產－淨額
不動產及設備－淨額
投資性不動產投資－淨額
無形資產－淨額
遞延所得稅資產
其他資產－淨額
資產總額
央行及銀行同業存款
央行及同業融資
透過損益按公允價值衡量之金融負債
避險之衍生金融負債－淨額
附買回票券及債券負債
應付款項
當期所得稅負債
與待出售資產直接相關之負債
存款及匯款
應付金融債券
應付公司債
特別股負債
其他金融負債
負債準備
遞延所得稅負債
其他負債
負債總額
股本
資本公積
保留盈餘
其他權益
庫藏股票
歸屬於母公司業主之權益合計
共同控制下前手權益
合併前非屬共同控制股權
非控制權益
權益總額
母公司暨子公司所持有之母公司庫藏股股數（單位：股）
預收股款（權益項下）之約當發行股數（單位：股）
每股參考淨值
流動資產
非流動資產
資產合計
流動負債
非流動負債
負債合計
保留盈餘（或累積虧損）
歸屬於母公司業主權益合計
權益合計
母公司暨子公司持有之母公司庫藏股股數（單位：股）
存放央行及拆借金融同業
避險之衍生金融資產
附賣回票券及債券投資
再保險合約資產－淨額
投資性不動產－淨額
央行及金融同業存款
避險之衍生金融負債
應付商業本票－淨額
應付債券
其他借款
歸屬於母公司業主之權益
應收款項
待出售資產
投資
再保險合約資產
不動產及設備
無形資產
其他資產
分離帳戶保險商品資產
短期債務
以成本衡量之金融負債
保險負債
具金融商品性質之保險契約準備
外匯價格變動準備
分離帳戶保險商品負債
庫藏股
年份
季度
公司名稱
待註銷股本股數（單位：股）
待分配予業主之資產－淨額
透過其他綜合損益按公允價值衡量之金融資產
按攤銷後成本衡量之債務工具投資
資產總計
負債總計
權益總計
Unnamed: 13
本期所得稅資產
本期所得稅負債
使用權資產－淨額
租賃負債
Unnamed: 12
待分配予業主之資產（或處分群組）
使用權資產


,公司代號,現金及約當現金,存放央行及拆借銀行同業,透過損益按公允價值衡量之金融資產,避險之衍生金融資產淨額,附賣回票券及債券投資淨額,應收款項－淨額,當期所得稅資產,待出售資產－淨額,貼現及放款－淨額,...,負債總計,權益總計,Unnamed: 13,本期所得稅資產,本期所得稅負債,使用權資產－淨額,租賃負債,Unnamed: 12,待分配予業主之資產（或處分群組）,使用權資產
